In [1]:
#ссылка на мой репозиторий в git hub: https://github.com/angelica-t/HSE-Machine-Learning
import pandas as pd
import os
import json
import numpy as np
from pandas.io.json import json_normalize


#some_data=data.sample(frac=0.1, random_state=2)
#print(some_data.dtypes)
data = pd.read_csv("./train.csv", low_memory=False)
df = data.sample(frac=0.1, random_state=2)
df_s = data.sample(frac=0.01, random_state=2)

df.to_csv(r'10%_data.csv', index = False)
df_s.to_csv(r'1%_data.csv', index = False)

In [2]:
def load_df(csv_path='./1%_data.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
df_train = load_df()

Loaded 1%_data.csv. Shape: (9037, 54)


In [128]:

print(df_train.dtypes)

channelGrouping                       object
date                                   int64
fullVisitorId                         object
sessionId                             object
socialEngagementType                  object
visitId                                int64
visitNumber                            int64
visitStartTime                datetime64[ns]
device.browser                        object
device.operatingSystem                object
device.isMobile                         bool
device.deviceCategory                 object
geoNetwork.continent                  object
geoNetwork.subContinent               object
geoNetwork.country                    object
geoNetwork.networkDomain              object
totals.visits                         object
totals.hits                           object
totals.pageviews                     float64
totals.newVisits                      object
trafficSource.source                  object
trafficSource.medium                  object
Revenues f

In [127]:
revenue=getcol['totals.transactionRevenue'].fillna(0.0).astype(float)
df_train['Revenues from transactions']=revenue


C:\Users\tagil\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [121]:
from datetime import datetime
df_train['visitStartTime']=pd.to_datetime(df_train['visitStartTime'], unit='s')
df_train['totals.pageviews']=df_train['totals.pageviews'].fillna(0.0).astype(float)
browser=df_train['device.browser'].astype('category')
operSystem=df_train['device.operatingSystem'].astype('category')
subcont=df_train['geoNetwork.subContinent'].astype('category')
channel=df_train['channelGrouping'].astype('category')
engagedOrNot=df_train['socialEngagementType'].astype('category')

In [132]:
# ГРУППИРОВКА И ФУНКЦИИ
from math import log1p
def agg(group): # функция, в которой считаются все показатели
    freeday=0
    daytime=0
    revenueForUser=log1p((group['Revenues from transactions']).sum())
    pageViewsForUser=sum(group['totals.pageviews'])
    numberOfDevices=group['device.operatingSystem'].nunique()
    for index, row in group.iterrows(): # пробегаемся по датам в каждой группе 
        if row['visitStartTime'].strftime('%A')=='Saturday' or row['visitStartTime'].strftime('%A')=='Sunday':
            freeday=1 #выходной день или нет
        if 5<=row['visitStartTime'].hour<=21:
            daytime=1 #дневное или ночное время
            
    return freeday, daytime, revenueForUser, pageViewsForUser, numberOfDevices
def func(group): #функция, в которой все посчитанные показатели собираются в один массив
    freeday,daytime, revenueForUser,pageViewsForUser,numberOfDevices=agg(group)
    d = {}
    d['Weekend or not'] = freeday
    d['Daytime or not'] = daytime
    d['Revenue']=revenueForUser
    d['Page views']=pageViewsForUser
    d['Number of devices']=numberOfDevices
    return pd.Series(d, index=['Weekend or not','Daytime or not','Revenue','Page views','Number of devices'])
groupedData = df_train.groupby("fullVisitorId").apply(func)
groupedData= groupedData.reset_index()
groupedData['Browser']=browser.cat.codes
groupedData['Operating system']=operSystem.cat.codes
groupedData['Part of the world']=subcont.cat.codes
groupedData['Channel types']=channel.cat.codes
groupedData['Socially engaged']=engagedOrNot.cat.codes
groupedData

,fullVisitorId,Weekend or not,Daytime or not,Revenue,Page views,Number of devices,Browser,Operating system,Part of the world,Channel types,Socially engaged
0,000005103959234087,1.0,0.0,0.0,8.0,1.0,5,9,17,7,0
1,0003546642773393141,1.0,1.0,0.0,1.0,1.0,5,9,6,4,0
2,0005627692133034833,0.0,1.0,0.0,1.0,1.0,5,9,7,7,0
3,0007194843978514987,0.0,1.0,0.0,1.0,1.0,19,6,14,7,0
4,0007514108108860221,0.0,1.0,0.0,1.0,1.0,5,9,11,4,0
...,...,...,...,...,...,...,...,...,...,...,...
8967,9997870904571299374,0.0,0.0,0.0,1.0,1.0,19,12,11,2,0
8968,9998025013142890030,0.0,0.0,0.0,1.0,1.0,5,9,7,2,0
8969,9998030024650532978,0.0,0.0,0.0,1.0,1.0,5,9,11,4,0
8970,9998650597961901505,1.0,0.0,0.0,1.0,1.0,5,3,11,4,0


In [140]:
#  бустинг
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
X=groupedData.drop('Revenue',axis=1) # исключаем из данных зависимую переменную
y=groupedData['Revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

gb = GradientBoostingRegressor(max_depth=30,
                               n_estimators=20,
                               random_state=2)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import confusion_matrix, classification_report
# Compute MSE
mse_test = MSE(y_test, y_pred)
# Compute RMSE
rmse_test = mse_test**(1/2)
from sklearn.model_selection import cross_val_score
# Compute 5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(gb, X,y,cv=5)
# Print the 5-fold cross-validation scores
print('Test set RMSE of gb: {:.3f}'.format(rmse_test))
print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

Test set RMSE of gb: 2.656
5-fold cross-validation scores
[-0.26182307 -0.47594073 -0.47227241 -0.37652664 -0.02409623]
Average 5-Fold CV Score
-0.32213181413382014


In [141]:
# Случайный лес
# Import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
# Instantiate rf
rf = RandomForestRegressor(n_estimators=25,
                           random_state=2)
                         
# Fit rf to the training set            
rf.fit(X_train, y_train)   
# Predict the test set labels
y_pred = rf.predict(X_test)
# Evaluate the test set RMSE
rmse_test = MSE(y_test, y_pred)**(1/2)
# Print rmse_test
print('Test set RMSE of rf: {:.2f}'.format(rmse_test))
# Compute 5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(rf, X,y,cv=5)
# Print the 5-fold cross-validation scores

print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

Test set RMSE of rf: 2.03
5-fold cross-validation scores
[ 0.09513643 -0.02813583 -0.03495073  0.02796471  0.17813366]
Average 5-Fold CV Score
0.04762964715546927


In [143]:
# SVM
from sklearn.svm import SVR
SVMreg = SVR(kernel='rbf')
SVMreg.fit(X,y)
print('SVM Score:')
print(SVMreg.score(X_train, y_train))
print(SVMreg.score(X_test, y_test))
# Compute 5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(SVMreg, X,y,cv=5)
# Print the 5-fold cross-validation scores
print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM Score:
-7.575872138509965e-07
-7.042016490022007e-05


C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

5-fold cross-validation scores
[-0.00434687  0.00253511 -0.00377188 -0.00049322 -0.00259551]
Average 5-Fold CV Score
-0.0017344763386563321


In [146]:
# линейная регрессия
# Import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Create the regressor: reg
reg = LinearRegression()
reg.fit(X_train,y_train)
# Predict on the test data: y_pred
y_pred = reg.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2: {}".format(reg.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error: {}".format(rmse))
# Compute 5-fold cross-validation scores: cv_scores
cv_scores = cross_val_score(SVMreg, X,y,cv=5)
# Print the 5-fold cross-validation scores
print('5-fold cross-validation scores')
print(cv_scores)
print("Average 5-Fold CV Score")
print(np.mean(cv_scores))

R^2: -3.795865853239988e-05
Root Mean Squared Error: 2.1702079916435753


C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

5-fold cross-validation scores
[-0.00434687  0.00253511 -0.00377188 -0.00049322 -0.00259551]
Average 5-Fold CV Score
-0.0017344763386563321
